In [28]:
import psycopg2
from datetime import date, timedelta

def get_optimal_params(start_date, end_date, conn):
    """
    Retrieve the optimal parameters between start_date and end_date.
    """
    query = """
        SELECT threshold, threshold_abs, discount_rate
        FROM (
            SELECT threshold, threshold_abs, discount_rate, SUM(daily_profit) as total_profit
            FROM (
                SELECT threshold, threshold_abs, discount_rate, day, SUM(profit) as daily_profit
                FROM revenues_1_cycle_max_h
                WHERE day BETWEEN %s AND %s
                GROUP BY day, threshold, threshold_abs, discount_rate
            ) AS day_aggregated
            GROUP BY threshold, threshold_abs, discount_rate
        ) AS param_aggregated
        ORDER BY total_profit DESC
        LIMIT 1;
    """
    with conn.cursor() as cursor:
        cursor.execute(query, (start_date, end_date))
        result = cursor.fetchone()
        return result

def get_profit_with_params(start_date, end_date, threshold, threshold_abs, discount_rate, conn):
    """
    Retrieve the total profit between start_date and end_date using given trading parameters.
    """
    query = """
        SELECT SUM(daily_profit)
        FROM (
            SELECT day, SUM(profit) as daily_profit
            FROM revenues_1_cycle_max_h
            WHERE day BETWEEN %s AND %s AND threshold = %s AND threshold_abs = %s AND discount_rate = %s
            GROUP BY day
        ) AS aggregated;
    """
    with conn.cursor() as cursor:
        cursor.execute(query, (start_date, end_date, threshold, threshold_abs, discount_rate))
        result = cursor.fetchone()
        # Check if result is None and return 0, otherwise return the result
        if result is None:
            print("Result is None")

        return result[0] if result and result[0] is not None else 0


def simulate_trading(number_of_days, year, conn):
    start_date = date(year - 1, 12, 31) - timedelta(days=number_of_days-1)
    end_date = start_date + timedelta(days=number_of_days-1)
    last_day_of_year = date(year, 12, 31)

    total_profit = 0

    while end_date <= last_day_of_year:
        # Find optimal parameters for the last `number_of_days` days
        threshold, threshold_abs, discount_rate = get_optimal_params(start_date, end_date, conn)

        # Compute profit for the next `number_of_days` days using optimal parameters
        profit_start_date = end_date + timedelta(days=1)
        profit_end_date = profit_start_date + timedelta(days=number_of_days-1)
        total_profit += get_profit_with_params(profit_start_date, profit_end_date, threshold, threshold_abs, discount_rate, conn)

        # Slide the window
        start_date = profit_start_date
        end_date = profit_end_date

    return total_profit


CONNECTION = ""
conn = psycopg2.connect(CONNECTION)
cursor = conn.cursor()
cursor.execute("ROLLBACK")

profit = simulate_trading(1, 2022, conn)
print(profit)

66161.58021699998
